In [1]:
%pip install openai httpx --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00


In [2]:
# Get the openai secret key:
import getpass
from openai import OpenAI
import re
import httpx

secret_key = getpass.getpass("Please enter your openai key: ")

Please enter your openai key: ··········


In [4]:
# This code is adapted from Simon Willison https://til.simonwillison.net/llms/python-react-pattern
client = OpenAI(api_key = secret_key)
client.api_key = secret_key

class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(model="gpt-3.5-turbo", messages=self.messages)
        # print(completion.usage)
        return completion.choices[0].message.content

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
""".strip()


action_re = re.compile('^Action: (\w+): (.*)$')

def query(question, max_turns=5):
    i = 0
    bot = ChatBot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]

def calculate(what):
    return eval(what)

known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
}

This code is designed to create an interactive chatbot using OpenAI's `gpt-3.5-turbo` model, which follows a specific reasoning pattern (Thought, Action, PAUSE, Observation) to answer user questions.
### Key Elements:
1. **API Initialization**:
   - The OpenAI client is initialized with a secret API key.
   - `client = OpenAI(api_key=secret_key)` creates an OpenAI client, enabling communication with the API.

2. **`ChatBot` Class**:
   - **Initialization** (`__init__`):
     - This initializes a chatbot with an optional system message that sets the context for the conversation. If the system message is provided, it is stored in `self.messages` with the role `"system"`.
   - **`__call__` Method**:
     - When the chatbot instance is called with a user message, the message is added to the `self.messages` list with the role `"user"`.
     - It then calls `execute()` to get a response from the model, which is added to the messages under the role `"assistant"`. The response is returned.
   - **`execute()` Method**:
     - This method communicates with OpenAI’s API using the `client.chat.completions.create()` method, passing the chat history (`self.messages`). It returns the content of the response from the model.

3. **Prompt Setup**:
   - The `prompt` variable provides instructions to the chatbot, setting up a pattern for reasoning:
     - **Thought**: The chatbot thinks about the question.
     - **Action**: It may perform actions like calculating or looking things up on Wikipedia.
     - **PAUSE**: A placeholder to indicate it needs to wait for additional information or execution of an action.
     - **Observation**: It observes the result of the action and continues processing to form the final answer.

4. **Action Recognition**:
   - The `action_re` regular expression (`Action: (\w+): (.*)`) is used to identify actions within the chatbot's response.
   - Actions like `calculate` and `wikipedia` are available to the chatbot, and these actions are executed in response to certain triggers in the response.

5. **Main Logic - `query()` Function**:
   - This function initiates the conversation with the chatbot. It loops over multiple turns (up to `max_turns`), sending the user question and handling the chatbot’s responses.
   - If the response contains an action (recognized via the regular expression), the corresponding function (like `wikipedia()` or `calculate()`) is called, and the chatbot continues processing with the result.
   - The loop continues until the chatbot provides an answer, with `Thought`, `Action`, `PAUSE`, and `Observation` guiding the conversation flow.

6. **Known Actions**:
   - Two action handlers are defined:
     - **`wikipedia()`**: This function sends a query to Wikipedia's API and returns a snippet of the search result.
     - **`calculate()`**: This function evaluates mathematical expressions using Python's `eval()`.

### Example:
In a conversation, if the chatbot is asked, "What is the capital of France?", the flow would be:
1. **Thought**: "I should look up France on Wikipedia."
2. **Action**: `wikipedia: France`
3. **PAUSE**: The bot waits for the action to be executed (i.e., it queries Wikipedia).
4. **Observation**: The chatbot observes the response from Wikipedia ("The capital is Paris").
5. **Answer**: "The capital of France is Paris."

### Error Handling:
If an unknown action is encountered, the program will raise an exception:
```python
raise Exception("Unknown action: {}: {}".format(action, action_input))
```

This pattern mimics a human-like reasoning process, making the chatbot more interactive and capable of handling more complex tasks like calculations or fetching information from external sources.

In [6]:
query("What is the capital of england?")

Thought: I should look up England on Wikipedia.
Action: wikipedia: England
PAUSE
 -- running wikipedia England
Observation: <span class="searchmatch">England</span> is a country that is part of the United Kingdom. It is located on the island of Great Britain, of which it covers approximately 62%, and over
Answer: The capital of England is London.


In [9]:
query("What is 2 + 9?")

Thought: I can easily calculate the sum of 2 + 9.

Action: calculate: 2 + 9
PAUSE
 -- running calculate 2 + 9
Observation: 11
Answer: The sum of 2 + 9 is 11
